In [ ]:
# type: ignore
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plot
%matplotlib inline

In [32]:
words = open('data/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [33]:
len(words)

32033

In [34]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [35]:
# dataset creation
context_length = 3
# , labels
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * context_length
    
    for char in w + '.':
        ind = stoi[char]
        X.append(context)
        Y.append(ind)
        print(''.join(itos[i] for i in context), '-->', itos[ind])
        context = context[1:] + [ind]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... --> e
..e --> m
.em --> m
emm --> a
mma --> .
olivia
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
ava
... --> a
..a --> v
.av --> a
ava --> .
isabella
... --> i
..i --> s
.is --> a
isa --> b
sab --> e
abe --> l
bel --> l
ell --> a
lla --> .
sophia
... --> s
..s --> o
.so --> p
sop --> h
oph --> i
phi --> a
hia --> .


In [43]:
X.shape, X.dtype, Y.shape, Y.dtype # type: ignore

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [44]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [37]:
# create 2-dimensional embeddings for each character
C = torch.randn((27, 2))
print(C.shape)
C

torch.Size([27, 2])


tensor([[ 1.8121e+00, -1.0774e+00],
        [ 7.1450e-01,  1.3835e+00],
        [ 9.6207e-01,  1.7333e-01],
        [ 7.5742e-01,  5.9888e-01],
        [ 1.4570e+00, -2.9258e-01],
        [-9.8830e-01,  9.2055e-01],
        [-7.3107e-01, -8.6211e-01],
        [ 2.7626e-02, -2.0563e-01],
        [-2.4009e+00, -7.9040e-01],
        [ 1.1729e-01, -2.5543e-01],
        [-1.0108e+00, -9.9343e-01],
        [-1.7799e+00,  1.6636e+00],
        [-7.9049e-01, -7.4993e-01],
        [-1.4935e-01,  1.0707e+00],
        [-2.9669e-01,  2.1949e+00],
        [ 8.3863e-01,  1.1487e+00],
        [ 1.0149e-01, -4.7976e-01],
        [ 6.9843e-01, -1.8436e-01],
        [-4.1806e-01,  1.1658e+00],
        [ 5.2739e-01, -8.8634e-01],
        [-1.6801e+00,  6.6278e-01],
        [ 2.2071e+00,  5.9645e-01],
        [ 2.6439e-01, -3.3268e-01],
        [ 2.8381e-01,  3.0942e-01],
        [-1.0442e-01, -6.2841e-01],
        [ 1.7937e-03,  7.2023e-01],
        [ 1.3094e+00,  1.6723e+00]])

In [38]:
# the embedding of 'd'
C[5]

tensor([-0.9883,  0.9205])

In [39]:
# Note that this is equivalent to the above cell
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.9883,  0.9205])

In [40]:
# we can also index into C using the entire X tensor
# this has replaced each index in X with its embedding
emb = C[X]
C[X].shape

torch.Size([32, 3, 2])

![Bengio et al. Architecture](images/bengioetal.png)

In [42]:
# first hidden layer
# 6 (each context piece has 3 embeddings of size 2) inputs to each of 100 (arbitrary) neurons
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [66]:
# emb.shape => (32, 3, 2) and W1 => (6, 100)
hidden_layer_one = emb.view(emb.shape[0], 6) @ W1 + b1
# tanh => [0, 1]
hidden_layer_one = torch.tanh(activations)
hidden_layer_one.shape

torch.Size([32, 100])

In [67]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [70]:
# logits === final outputs
logits = hidden_layer_one @ W2 + b2
logits.shape

torch.Size([32, 27])

### Softmax!

![Softmax in Neural Nets](images/softmax.png)

In [77]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
print(prob[0].sum())
prob.shape

tensor(1.)


torch.Size([32, 27])

In [79]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [78]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])